geopandas installed easily with "pip install geopandas" using python 3.7 the other dependencies, shapely,fiona etc are dependencies of geopandas and should install with the geopandas. 

In [3]:
import pandas as pd
import requests
# import geopandas as gpd
# import fiona
# from shapely.geometry import Point, mapping,shape
# from geopandas import GeoDataFrame, read_file
#from geopandas.tools import overlay
# from geopandas.tools import sjoin
import datetime as dt

In [4]:

#list of filenames to be called when reading in    
filelist = ['COMED','DAYTON']   
  
## to be called when reading in json files    
def cleandata(df):
    df.insert(3,'Date_Clean', df.dt_iso.apply(lambda x: x[:19])
                       .apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')))
    df.insert(4,"Year", df.Date_Clean.dt.year)      
    df.insert(5,"Month", df.Date_Clean.dt.month)
    df.insert(6,"Day",df.Date_Clean.dt.day)
    df.insert(7,"Day_of_Week",df.Date_Clean.dt.weekday)
    df.insert(8,"Weekend",df.Date_Clean.dt.weekday.apply(lambda x:1 if x > 4 else 0))
    df['Temp(f)'] = [round((df['main'][x]['temp_max']-273.15)*(9/5)+32,2) for x in range(len(df.main.values))]
    

    return  df


In [5]:
# Reading in Power Data csv's
dfPwr = pd.DataFrame()

for x in range(len(filelist)):
    df = pd.read_csv(f'{filelist[x]}_hourly.csv',parse_dates = ['Datetime'])
    df.rename(columns = {f'{filelist[x]}_MW': 'MW'},inplace = True)
    df.insert(2,'UTILITY',filelist[x])
    
    dfPwr = dfPwr.append(df,ignore_index= True,sort = False)
dfPwr = dfPwr[(dfPwr.Datetime >= '2015-01-01') & (dfPwr.Datetime < '2018-01-01') ].copy()    

In [6]:
# Reading in Weather data jsons 
weatherDf = pd.DataFrame()
for x in range(len(filelist)):
    wdf = pd.read_json(f'{filelist[x]}_Weather.json')
    wdf.insert(0,'Utility',filelist[x]) 
    # Calling function defined above
    wdf = cleandata(wdf)

    weatherDf = weatherDf.append(wdf,ignore_index = True,sort = False)

weatherDf = weatherDf[['Utility','Date_Clean','Year','Month','Day','Day_of_Week',
                      'Weekend','Temp(f)']].copy()
weatherDf = weatherDf[(weatherDf.Date_Clean >= '2015-01-01') & (weatherDf.Date_Clean < '2018-01-01') ].copy()

In [7]:
# merging power and weather data and housekeeping
weatherPwrDf = dfPwr.merge(weatherDf,how = 'inner',left_on = ['Datetime','UTILITY'],right_on = ['Date_Clean','Utility'])
weatherPwrDf = weatherPwrDf[['Datetime','MW','Utility','Year','Month','Day','Day_of_Week','Weekend','Temp(f)']].copy()

In [13]:
df2016 = weatherPwrDf[weatherPwrDf.Year == 2016].copy()
df2016.head()

,Datetime,MW,Utility,Year,Month,Day,Day_of_Week,Weekend,Temp(f)
3593,2016-12-31 01:00:00,10419.0,COMED,2016,12,31,5,1,32.0
3594,2016-12-31 02:00:00,9940.0,COMED,2016,12,31,5,1,32.0
3595,2016-12-31 03:00:00,9604.0,COMED,2016,12,31,5,1,35.6
3596,2016-12-31 04:00:00,9318.0,COMED,2016,12,31,5,1,37.4
3597,2016-12-31 05:00:00,9224.0,COMED,2016,12,31,5,1,37.4


In [14]:
df2016.to_csv('2016_Data')

Code below is simply to get geographic data for visualization in tableau, no need to join to it until ML is completed 

In [8]:
gpd.pd.set_option('display.max_columns',500)

## reading in shape file
geoFile = gpd.GeoDataFrame.from_file('Electric_Retail_Service_Territories.shp')
## resetting names of locations for later joins 
geoFile.loc[(geoFile.NAME.str.contains('EDISON'))& (geoFile.STATE == 'IL'),'NAME'] ='COMED'
geoFile.loc[geoFile.NAME ==('DAYTON POWER & LIGHT CO'),'NAME'] = 'DAYTON'
#limiting the file size for geographic area of interest
geoFile = geoFile[geoFile.CNTRL_AREA == 'PJM INTERCONNECTION, LLC'].copy()

## Join to be done after ML completed 
#geoPowerdf = geoFile.merge(weatherPwrDf,how = 'left',left_on = 'NAME',right_on = 'Utility')

NameError: name 'gpd' is not defined

In [5]:
geoFile.to_file('Electric_Retail_Service_Territories.shp')